In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer #for numerical data
from sklearn.preprocessing import OneHotEncoder #for nominal data
from sklearn.preprocessing import OrdinalEncoder #for ordinal data

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.sample(4)

,age,gender,fever,cough,city,has_covid
26,19,Female,100.0,Mild,Kolkata,Yes
28,16,Male,104.0,Mild,Kolkata,No
72,83,Female,101.0,Mild,Kolkata,No
93,27,Male,100.0,Mild,Kolkata,Yes


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [8]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [10]:
X_train

,age,gender,fever,cough,city
96,51,Female,101.0,Strong,Kolkata
7,20,Female,NaN,Strong,Mumbai
77,8,Female,101.0,Mild,Kolkata
84,69,Female,98.0,Strong,Mumbai
37,55,Male,100.0,Mild,Kolkata
...,...,...,...,...,...
68,54,Female,104.0,Strong,Kolkata
70,68,Female,101.0,Strong,Delhi
28,16,Male,104.0,Mild,Kolkata
98,5,Female,98.0,Strong,Mumbai


# 1. Aam Zindegi

In [15]:
#adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [16]:
#ordinalEncoding --> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.transform(X_test[['cough']])

X_test_cough.shape

(20, 1)

In [18]:
#oneHotEncoding --> gender,city {Nominal}
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

# 2gender+4city
# remove the frst column...
#we got total 4 column

(80, 4)

In [19]:
#extracting age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

#also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [20]:
X_train_age.shape

(80, 1)

In [21]:
X_train_transformed = np.concatenate((X_train_age,X_train_cough,X_train_fever,X_train_gender_city),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_cough,X_test_fever,X_test_gender_city),axis =1)

In [22]:
X_train_transformed.shape

(80, 7)

### life will get easier with columnTransform
# 2. Mentos Zindege

In [23]:
from sklearn.compose import ColumnTransformer

In [25]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse=False),['gender','city'])
], remainder='passthrough')
# transformers = list of (name, transformer, columns) tuples

In [27]:
transformer.fit_transform(X_train).shape

(80, 7)

In [28]:
transformer.transform(X_test).shape

(20, 7)